# Temperature Control

Import necessary packages

In [1]:
from distutils.log import error
import board
import digitalio
import adafruit_max31856
import time
from datetime import datetime as dt
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np
import keyboard
from tclab import clock, setup, Historian, Plotter
import PID
import os
from IPython.display import clear_output

Get current directory so we can save log files in a separate folder.

In [2]:
ROOT_DIR = os.path.realpath(os.path.join(os.path.dirname("Temperature Control.ipynb")))

Define the temperature and error logging functions that output to csv and text files respectivly.

In [3]:
def log_error(f_name,error,time,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write('')
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write("\n".join('{} at t= {}'.format(error,str(time))))

def log_temps(f_name,header,data,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(header)
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(data)

Create the temperature and error log files.

In [4]:
data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
data_header=['Rt', 'temp_ch', 'temp_hex_f', 'temp_hex_b', 'temp_chamber', 'MV1','Heater17 On', 'MV2','Heater18 On']
log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0],True)
updated=False

Define the coldhead, heat exchanger, chamber, and heater entities that will be controled.

In [5]:
# Create sensor object, communicating over the board's default SPI bus
spi = board.SPI()

# allocate a CS pin and set the direction
cs13 = digitalio.DigitalInOut(board.D13)
cs13.direction = digitalio.Direction.OUTPUT
cs25 = digitalio.DigitalInOut(board.D25)
cs25.direction = digitalio.Direction.OUTPUT
cs26 = digitalio.DigitalInOut(board.D26)
cs26.direction = digitalio.Direction.OUTPUT
cs16 = digitalio.DigitalInOut(board.D16)
cs16.direction = digitalio.Direction.OUTPUT
HeaterF = digitalio.DigitalInOut(board.D17)
HeaterF.direction= digitalio.Direction.OUTPUT
HeaterB = digitalio.DigitalInOut(board.D18)
HeaterB.direction = digitalio.Direction.OUTPUT


# create a thermocouple object with the above
ColdHead = adafruit_max31856.MAX31856(spi, cs25,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
HeatExF = adafruit_max31856.MAX31856(spi, cs13,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
HeatExB = adafruit_max31856.MAX31856(spi, cs16,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
Chamber = adafruit_max31856.MAX31856(spi, cs26,thermocouple_type=adafruit_max31856.ThermocoupleType.T)


Initiate PID controls for Front Heat Exchanger

In [6]:
HeaterF.value = False
HeatF_on=True
targetT1 = -105    #initial inputs for PID control
P1 = 2500
I1 = 2
D1 = 0

controllerF = PID.PID(P1, I1, D1)        # create pid control
controllerF.SetPoint = targetT1             # initialize
controllerF.setSampleTime(3)

Initiate PID controls for Back Heat Exchanger

In [7]:
HeaterB.value = False 
HeatB_on = True
targetT2 = -105
P2 = 2500
I2 = 2
D2 = 0

controllerB = PID.PID(P2, I2, D2)
controllerB.SetPoint = targetT2
controllerB.setSampleTime(3)

Read the coldhead, heat exhanger and chamber temperatures then log them.

In [8]:
temp_coldhead=ColdHead.temperature
temp_HeatExF=HeatExF.temperature
temp_HeatExB=HeatExB.temperature
temp_chamber=Chamber.temperature

log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, controllerF.output,"False", controllerB.output, "False"],False)

Create the temperature plot

In [9]:
plot_window = 10
coldhead_temps = np.array(np.zeros([plot_window]))
HeatExF_temps = np.array(np.zeros([plot_window]))
HeatExB_temps = np.array(np.zeros([plot_window]))
chamber_temps = np.array(np.zeros([plot_window]))
time_stamps = []
#np.array(np.zeros([plot_window]))
for i in range(plot_window):
    time_stamps.append(dt.now().strftime('%H:%M:%S'))
    #x_var.append(time.asctime(time.time()))
plt.ion()
fig, ax = plt.subplots()
chline,  = ax.plot(time_stamps, coldhead_temps, label='Cold Head')
hexfline, = ax.plot(time_stamps, HeatExF_temps, label='Heat Exchanger Front')
hexbline, = ax.plot(time_stamps, HeatExB_temps, label='Heat Exchanger Back')
chamberline, =ax.plot(time_stamps, chamber_temps, label='Chamber')
plot_window = 3000
ax.locator_params(tight=True, nbins=4)
ax.legend()
ax.set_xlabel('Time (Hour:Min)')
ax.set_ylabel('Temperature (Celsius)')

/tmp/ipykernel_28796/643317237.py:18: UserWarning: 'set_params()' not defined for locator of type <class 'matplotlib.category.StrCategoryLocator'>
  ax.locator_params(tight=True, nbins=4)


Text(0, 0.5, 'Temperature (Celsius)')

Loop that reads the temperatures of the coldhead, heat exhanger and chamber logs them, updates the PID control and runs the heater, then plots the tmperatures.

In [ ]:
print_log=[["Time" ,"Coldhead", "Heat Exchanger Front", "Heat Exchanger Back", "Chamber", "PID1 Output", "Heater17 Status", "PID2 Output", "Heater18 Status"]]
while True:
    now = time.time()
    clear_output(wait=True)
    if os.stat(data_f_name).st_size>= 4194304:
            data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
            updated=True
    temp_coldhead=ColdHead.temperature
    temp_HeatExF=HeatExF.temperature
    temp_HeatExB=HeatExB.temperature
    temp_chamber=Chamber.temperature
    controllerF.update(temp_HeatExF) # compute manipulated variable
    controllerB.update(temp_HeatExB)
    MV1 = controllerF.output # apply
    MV2 = controllerB.output
    if MV1 > 0:
        HeaterF.value = True
        HeatF_on=True
    else:
        HeaterF.value = False
        HeatF_on=False
    if MV2 > 0:
        HeaterB.value = True
        HeatB_on = True
    else:
        HeaterB.value = False
        HeatB_on = False
    #I really feel like there is a better way to do the following but I don't know what that would be 
    if updated:
        if HeatF_on and HeatB_on:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'On', MV2, 'On'],False)   #push the temperature readings to the log file
            updated=False
        elif HeatF_on and not HeatB_on:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'On', MV2, 'Off'],False)   #push the temperature readings to the log file
            updated=False
        elif not HeatF_on and HeatB_on:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'Off', MV2, 'On'],False)   #push the temperature readings to the log file
            updated=False
        else:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'Off', MV2, 'Off'],False)
            updated=False
    else:
        if HeatF_on and HeatB_on:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'On', MV2, 'On'],False) #push the temperature readings to the log file
        elif HeatF_on and not HeatB_on:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'On', MV2, 'Off'],False)
        elif not HeatF_on and HeatB_on:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'Off', MV2, 'On'],False)
        else:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, 'Off', MV2, 'Off'],False)
    if len(print_log)<=21:
        if HeatF_on and HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "On", MV2, 'On'])
        elif HeatF_on and not HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "On", MV2, 'Off'])
        elif not HeatF_on and HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "Off", MV2, 'On'])
        else:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "Off", MV2, 'Off'])
    else:
        print_log.pop(1)
        if HeatF_on and HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "On", MV2, 'On'])
        elif HeatF_on and not HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "On", MV2, 'Off'])
        elif not HeatF_on and HeatB_on:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "Off", MV2, 'On'])
        else:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, "Off", MV2, 'Off'])
    for line in print_log:
        print(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],sep=" | ")
    coldhead_temps = np.append(coldhead_temps, temp_coldhead)
    HeatExF_temps = np.append(HeatExF_temps, temp_HeatExF)
    HeatExB_temps = np.append(HeatExB_temps, temp_HeatExB)
    chamber_temps = np.append(chamber_temps, temp_chamber)
    time_stamps.append(dt.now().strftime('%H:%M:%S'))
    coldhead_temps = coldhead_temps[1: plot_window + 1]
    HeatExF_temps = HeatExF_temps[1:plot_window+1]
    HeatExB_temps = HeatExB_temps[1:plot_window+1]
    chamber_temps = chamber_temps[1:plot_window+1]
    time_stamps = time_stamps[1: plot_window + 1]
    chline.set_ydata(coldhead_temps)
    chline.set_xdata(time_stamps)
    hexfline.set_ydata(HeatExF_temps)
    hexfline.set_xdata(time_stamps)
    hexbline.set_ydata(HeatExB_temps)
    hexbline.set_xdata(time_stamps)
    chamberline.set_ydata(chamber_temps)
    chamberline.set_xdata(time_stamps)
    ax.relim()
    #ax.set_ylim(0,24)
    ax.autoscale_view()
    ax.set_xticks(time_stamps[::100])
    ax.set_xticklabels(time_stamps[::100])
    fig.canvas.draw()
    fig.canvas.flush_events()
    elapsed = time.time() - now  # how long was it running?
    time.sleep(4.-elapsed)

Time | Coldhead | Heat Exchanger Front | Heat Exchanger Back | Chamber | PID1 Output | Heater17 Status | PID2 Output | Heater18 Status
2022-10-04 14:11:56 | -123.0546875 | -60.65625 | -8.59375 | 18.7421875 | -110899.375 | Off | -241055.625 | Off
2022-10-04 14:12:00 | -123.09375 | -60.671875 | -8.5703125 | 18.671875 | -110860.3125 | Off | -241114.21875 | Off
2022-10-04 14:12:04 | -123.0546875 | -60.7890625 | -8.5703125 | 18.65625 | -110567.34375 | Off | -241114.21875 | Off
2022-10-04 14:12:08 | -123.1171875 | -60.9375 | -8.6953125 | 18.609375 | -110196.25 | Off | -240801.71875 | Off
2022-10-04 14:12:12 | -123.328125 | -60.84375 | -8.78125 | 18.765625 | -110430.625 | Off | -240586.875 | Off
2022-10-04 14:12:16 | -123.328125 | -61.0 | -8.7578125 | 18.7421875 | -110040.0 | Off | -240645.46875 | Off
2022-10-04 14:12:20 | -123.2265625 | -61.0078125 | -8.796875 | 18.671875 | -110020.46875 | Off | -240547.8125 | Off
2022-10-04 14:12:24 | -123.25 | -61.1328125 | -8.84375 | 18.765625 | -109707.9